# Predicting abalone age with Amazon SageMaker and the XGBoost algorithm

**Created by: [Tomas Beuzen](https://tomasbeuzen.github.io/). Hosted on [GitHub](xxx).**

---
## Contents
1. [Introduction](#1)
2. [Setup](#2)
3. [Training the XGBoost model ](#3)
4. [Create the model](#4)
5. [Create endpoint](#5)
6. [Validate the model for use](#6)
7. [Delete Endpoint](#7)


---
## 1. Introduction <a id=1></a>

This notebook demonstrates the use of Amazon SageMaker’s implementation of the XGBoost algorithm to train and host a regression model. It uses the classic abalone dataset which, the original version of which can be found [here](https://archive.ics.uci.edu/ml/datasets/abalone). Briefly, the number of "rings" present in an abalone shell is a proxy for the age of the abalone (age [years] = rings + 1.5). We aim to predict the age of abalone based on eight physical measurements. The data provided in the tutorial has been modified from the original, namely, the "Sex" characteristic has been one-hot-encoded and the "rings" target variable has had 1.5 added to it to represent the abalone age in years.

This notebook is modified after the example provided by Amazon [here](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/xgboost_abalone/xgboost_abalone.ipynb). It has been significantly stripped down and modified to provide a bare minimum example illustrating how to build and deploy a model using SageMaker.

---
## 2. Setup <a id=2></a>


This notebook was created and tested on an ml.m2.medium notebook instance. The following code sets up paths to the S3 bucket we stored our data in previously.

In [9]:
%%bash
export AWS_REGION=us-east-1

In [34]:
import os
import boto3
import re
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
print("Region name: {}".format(region))

bucket_name = "sagemakeruseast1"  # <-- insert your bucket name here
#bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region, bucket)


s3 = boto3.resource('s3')
try:
    #if  my_region == 'us-east-1':
    s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

Region name: us-east-1
S3 bucket created successfully


In [35]:
prefix = 'xgboost-as-a-built-in-algo'
bucket_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(bucket_path)

s3://sagemakeruseast1/xgboost-as-a-built-in-algo/output


In [36]:
# upload files

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'output/abalone_train/abalone_train.csv')).upload_file('data/abalone_train.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'output/abalone_validation/abalone_validation.csv')).upload_file('data/abalone_validation.csv')


---
## 3. Training the XGBoost model <a id=3></a>

The following cell loads the Amazon sagemaker xgboost docker image

In [37]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'xgboost', '0.90-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [30]:
#dir(container)

The following cell sets the parameters for, and executes training of the XGBoost model. You should not have to change any setting here (unless you want to change the name of the job which is clearly marked in a comment below). Training the model should take around 5 minutes. The code periodically pings the status of the job and prints the output.

In [38]:
%%time
import boto3
from time import gmtime, strftime

job_name = 'deploy-tutorial-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())  # <-- feel free to change the name of your job if you wish
print("Training job", job_name)

create_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": bucket_path
        # + "/xgboost-model"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.m5.2xlarge",
        "VolumeSizeInGB": 5
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "silent":"0",
        "objective":"reg:linear",
        "num_round":"50"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 3600
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": bucket_path + '/abalone_train',
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": bucket_path + '/abalone_validation',
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None"
        }
    ]
}


client = boto3.client('sagemaker', region_name=region)
client.create_training_job(**create_training_params)

import time

status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)
while status !='Completed' and status!='Failed':
    time.sleep(60)
    status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print(status)

Training job deploy-tutorial-2024-11-12-09-06-17
InProgress
InProgress
InProgress
Completed
CPU times: user 97.2 ms, sys: 0 ns, total: 97.2 ms
Wall time: 3min


If you see the message "Completed" that means training sucessfully completed and the output model was stored in the output path specified by `training_params['OutputDataConfig']` above.

---
## 4. Create the model <a id=4></a>
In order to set up hosting, we have to import the model from training to hosting. The cell below creates a SageMaker Model from the training output above.

In [39]:
%%time
import boto3
from time import gmtime, strftime

model_name=job_name + '-model'
print(model_name)

info = client.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

deploy-tutorial-2024-11-12-09-06-17-model
s3://sagemakeruseast1/xgboost-as-a-built-in-algo/output/deploy-tutorial-2024-11-12-09-06-17/output/model.tar.gz
arn:aws:sagemaker:us-east-1:596676454538:model/deploy-tutorial-2024-11-12-09-06-17-model
CPU times: user 23.7 ms, sys: 0 ns, total: 23.7 ms
Wall time: 724 ms


---
## 5. Create endpoint <a id=5></a>

Now that we've created a model we need to create a HTTPS endpoint where your machine learning model is available to provide inferences.

### Create endpoint configuration
SageMaker supports configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, we need to create an endpoint configuration which describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. In addition, and more relevant for the current tutorial, the endpoint configuration describes the instance type required for model deployment. 

In [40]:
from time import gmtime, strftime

endpoint_config_name = 'deploy-tutorial-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m5.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

deploy-tutorial-2024-11-12-09-14-28
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:596676454538:endpoint-config/deploy-tutorial-2024-11-12-09-14-28


### Create endpoint
Finally we will create the endpoint that serves up the model, using the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. It will take about 10 minutes to run the cell below and set up the endpoint.

In [41]:
%%time
import time

endpoint_name = 'deploy-tutorial-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
while status=='Creating':
    print("Status: " + status)
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

deploy-tutorial-2024-11-12-09-43-31
arn:aws:sagemaker:us-east-1:596676454538:endpoint/deploy-tutorial-2024-11-12-09-43-31
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:596676454538:endpoint/deploy-tutorial-2024-11-12-09-43-31
Status: InService
CPU times: user 102 ms, sys: 10.4 ms, total: 113 ms
Wall time: 4min 1s


---
## 6. Validate the model for use <a id=6></a>
Now that we've created the endpoint we can test that our model is available to perform inference. Let's try it out by making a single prediction which we call the "payload" in the cell below.

In [42]:
import math

features = 'length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,sex_I,sex_M'
payload = '0.41,0.325,0.1,0.3555,0.146,0.072,0.105,0,1'

runtime_client = boto3.client('runtime.sagemaker', region_name=region)
response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read()
result = result.decode("utf-8")
result = result.split(',')
result = [math.ceil(float(i)) for i in result]
print(features)
print(payload)
print (f'Prediction: {result[0]:.0f}')

length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,sex_I,sex_M
0.41,0.325,0.1,0.3555,0.146,0.072,0.105,0,1
Prediction: 10


## 7. Delete Endpoint <a id=7></a>
Once you are done using the endpoint, you can use the following to delete it. 

In [43]:
client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'a7109ae2-a4c1-4ea4-bcb5-85f8204431eb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a7109ae2-a4c1-4ea4-bcb5-85f8204431eb',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 12 Nov 2024 10:27:18 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}